<a href="https://colab.research.google.com/github/naseembabu/Deep-Learning-Codes/blob/main/Music_genre_classification_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Import all the required libraries.



In [11]:
!pip install librosa==0.6.0

In [12]:
#import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

Now convert the audio data files into PNG format images or basically extracting the Spectrogram for every Audio.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [14]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./drive/My Drive/Colab Notebooks/genres/{g}'):
        songname = f'./drive/My Drive/Colab Notebooks/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

Creating a header for our CSV file.

In [15]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

Extracting features from Spectrogram: 

In [18]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./drive/My Drive/Colab Notebooks/genres/{g}'):
        songname = f'./drive/My Drive/Colab Notebooks/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)[0]
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

Data preprocessing: 

In [19]:
data = pd.read_csv('dataset.csv')
data.head()# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Building an ANN model

In [21]:
model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=128)

Epoch 1/100
7/7 [==============================] - 2s 3ms/step - loss: 2.2627 - accuracy: 0.1374
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 1.9101 - accuracy: 0.3983
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 1.6636 - accuracy: 0.4040
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 1.4906 - accuracy: 0.4982
Epoch 5/100
7/7 [==============================] - 0s 2ms/step - loss: 1.3077 - accuracy: 0.5604
Epoch 6/100
7/7 [==============================] - 0s 2ms/step - loss: 1.1856 - accuracy: 0.5702
Epoch 7/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0868 - accuracy: 0.6464
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 1.0352 - accuracy: 0.6521
Epoch 9/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9571 - accuracy: 0.6959
Epoch 10/100
7/7 [==============================] - 0s 2ms/step - loss: 0.9018 - accuracy: 0.7183
Epoch 11/100
7/7 [===========